In [ ]:
import math
import random

def f(x):
    """Функция из варианта 1: f(x) = sin(x) - x^2/2"""
    return math.sin(x) - x**2 / 2

def create_table(m_plus_1, a, b, node_type='equal'):
    """Создание таблицы значений функции"""
    m = m_plus_1 - 1
    if node_type == 'equal':
        # Равноотстоящие узлы
        h = (b - a) / m
        nodes = [a + i * h for i in range(m_plus_1)]
    else:
        # Случайные узлы (гарантированно попарно различные)
        nodes = sorted(random.sample([a + i * (b - a) / 1000 for i in range(1001)], m_plus_1))
    
    table = [(z, f(z)) for z in nodes]
    return table

def sort_table_by_distance(table, x):
    """Сортировка таблицы по расстоянию до точки x"""
    return sorted(table, key=lambda item: abs(item[0] - x))

def lagrange_interpolation(sorted_table, n, x):
    """Интерполяция методом Лагранжа"""
    result = 0.0
    sum_coefficients = 0.0
    
    for i in range(n + 1):
        xi, yi = sorted_table[i]
        term = yi
        
        for j in range(n + 1):
            if i != j:
                xj, _ = sorted_table[j]
                term *= (x - xj) / (xi - xj)
        
        result += term
        sum_coefficients += term / yi if yi != 0 else 0
    
    return result, sum_coefficients

def newton_interpolation(sorted_table, n, x):
    """Интерполяция методом Ньютона"""
    # Создаем таблицу 
    x_values = [item[0] for item in sorted_table[:n + 1]]
    y_values = [item[1] for item in sorted_table[:n + 1]]
    
    # Инициализиция
    dd_table = [[0.0] * (n + 1) for _ in range(n + 1)]
    
    for i in range(n + 1):
        dd_table[i][0] = y_values[i]
    
    # Заполняем таблицу разделенных разностей
    for j in range(1, n + 1):
        for i in range(n + 1 - j):
            dd_table[i][j] = (dd_table[i + 1][j - 1] - dd_table[i][j - 1]) / (x_values[i + j] - x_values[i])
    
    result = dd_table[0][0]
    product = 1.0
    
    for i in range(1, n + 1):
        product *= (x - x_values[i - 1])
        result += dd_table[0][i] * product
    
    return result, dd_table

def main():
    print("=" * 60)
    print("Задача алгебраического интерполирования")
    print("Функция: f(x) = sin(x) - x^2/2")
    print("=" * 60)
    
    # Ввод параметров
    while True:
        try:
            m_plus_1 = int(input("Введите число значений в таблице (m+1): "))
            if m_plus_1 < 2:
                print("Число значений должно быть не менее 2")
                continue
            break
        except ValueError:
            print("Ошибка: введите целое число")
    
    while True:
        try:
            a = float(input("Введите левый конец отрезка a: "))
            b = float(input("Введите правый конец отрезка b: "))
            if a >= b:
                print("a должно быть меньше b")
                continue
            break
        except ValueError:
            print("Ошибка: введите число")
    
    node_type = input("Выберите тип узлов (equal - равноотстоящие, random - случайные): ").strip().lower()
    if node_type not in ['equal', 'random']:
        node_type = 'equal'
        print("Используются равноотстоящие узлы по умолчанию")
    
    table = create_table(m_plus_1, a, b, node_type)
    
    print("\nИсходная таблица значений функции:")
    print("k\tz_k\t\tf(z_k)")
    print("-" * 40)
    for i, (z, fz) in enumerate(table):
        print(f"{i}\t{z:.6f}\t{fz:.6f}")
    
    m = m_plus_1 - 1
    
    while True:
        print("\n" + "=" * 50)
        
        while True:
            try:
                x_point = float(input("Введите точку интерполирования x: "))
                break
            except ValueError:
                print("Ошибка: введите число")
        
        # Ввод степени многочлена
        while True:
            try:
                n = int(input(f"Введите степень интерполяционного многочлена n ≤ {m}: "))
                if n < 0:
                    print("Степень явл отрицательной")
                elif n > m:
                    print(f"Введено недопустимое значение n. Должно быть n ≤ {m}")
                else:
                    break
            except ValueError:
                print("введиЕ Целое число")
        
        sorted_table = sort_table_by_distance(table, x_point)
        
        '''print(f"\nОтсортированная таблица (ближайшие к x = {x_point}):")
        print("k\tz_k\t\tf(z_k)\t\tРасстояние")
        print("-" * 55)
        for i, (z, fz) in enumerate(sorted_table):
            distance = abs(z - x_point)
            print(f"{i}\t{z:.6f}\t{fz:.6f}\t{distance:.6f}")'''
        
        print(f"\nУзлы для построения P_{n}(x):")
        for i in range(n + 1):
            z, fz = sorted_table[i]
            print(f"x_{i} = {z:.6f}, f(x_{i}) = {fz:.6f}")
        
        # Интерполяция Лагранжа
        Pn_L, sum_coeff = lagrange_interpolation(sorted_table, n, x_point)
        fx = f(x_point)
        error_L = abs(fx - Pn_L)
        
        print(f"\nФорма Лагранжа:")
        print(f"P_{n}_L({x_point}) = {Pn_L:.15f}")
        print(f"f({x_point}) = {fx:.15f} сама функция")
        print(f"Абсолютная погрешность: |f(x) - P_n_L(x)| = {error_L:.15f}")
        print(f"Сумма коэффициентов : {sum_coeff:.15f}")
        
        # Интерполяция Ньютона
        Pn_N, dd_table = newton_interpolation(sorted_table, n, x_point)
        error_N = abs(fx - Pn_N)
        
        print(f"\nФорма Ньютона:")
        print(f"P_{n}_N({x_point}) = {Pn_N:.15f}")
        print(f"Абсолютная погрешность: |f(x) - P_n_N(x)| = {error_N:.15f}")
        
        print(f"\nОкончательная Разность между формами: |P_n_L - P_n_N| = {abs(Pn_L - Pn_N):.15f}")
        
        # Контрольные проверки
        if any(abs(x_point - z) < 1e-10 for z, _ in table):
            print("\n✓ Точка x совпадает с одним из узлов таблицы")
        
        if error_L < 1e-8 and error_N < 1e-8:
            print("✓ Погрешности близки к нулю")
        
        cont = input("\nНовые x и n? - yes or no: ").strip().lower()
        if cont not in ['y', 'yes', 'д', 'да']:
            break

if __name__ == "__main__":
    main()

Задача алгебраического интерполирования
Функция: f(x) = sin(x) - x^2/2


Введите число значений в таблице (m+1):  20
Введите левый конец отрезка a:  -1
Введите правый конец отрезка b:  0
Выберите тип узлов (equal - равноотстоящие, random - случайные):  equal



Исходная таблица значений функции:
k	z_k		f(z_k)
----------------------------------------
0	-1.000000	-1.341471
1	-0.947368	-1.260635
2	-0.894737	-1.180321
3	-0.842105	-1.100617
4	-0.789474	-1.021617
5	-0.736842	-0.943421
6	-0.684211	-0.866133
7	-0.631579	-0.789866
8	-0.578947	-0.714733
9	-0.526316	-0.640855
10	-0.473684	-0.568356
11	-0.421053	-0.497364
12	-0.368421	-0.428010
13	-0.315789	-0.360428
14	-0.263158	-0.294757
15	-0.210526	-0.231135
16	-0.157895	-0.169705
17	-0.105263	-0.110609
18	-0.052632	-0.053992
19	0.000000	0.000000



Введите точку интерполирования x:  3.5
Введите степень интерполяционного многочлена n ≤ 19:  19



Узлы для построения P_19(x):
x_0 = 0.000000, f(x_0) = 0.000000
x_1 = -0.052632, f(x_1) = -0.053992
x_2 = -0.105263, f(x_2) = -0.110609
x_3 = -0.157895, f(x_3) = -0.169705
x_4 = -0.210526, f(x_4) = -0.231135
x_5 = -0.263158, f(x_5) = -0.294757
x_6 = -0.315789, f(x_6) = -0.360428
x_7 = -0.368421, f(x_7) = -0.428010
x_8 = -0.421053, f(x_8) = -0.497364
x_9 = -0.473684, f(x_9) = -0.568356
x_10 = -0.526316, f(x_10) = -0.640855
x_11 = -0.578947, f(x_11) = -0.714733
x_12 = -0.631579, f(x_12) = -0.789866
x_13 = -0.684211, f(x_13) = -0.866133
x_14 = -0.736842, f(x_14) = -0.943421
x_15 = -0.789474, f(x_15) = -1.021617
x_16 = -0.842105, f(x_16) = -1.100617
x_17 = -0.894737, f(x_17) = -1.180321
x_18 = -0.947368, f(x_18) = -1.260635
x_19 = -1.000000, f(x_19) = -1.341471

Форма Лагранжа:
P_19_L(3.5) = -170512384.000000000000000
f(3.5) = -6.475783227689620 сама функция
Абсолютная погрешность: |f(x) - P_n_L(x)| = 170512377.524216771125793
Сумма коэффициентов : -4821974577639078912.000000000000000

Фор

...


Новые x и n? - yes or no:  y


Введите точку интерполирования x:  1
Введите степень интерполяционного многочлена n ≤ 19:  


введиЕ Целое число


Введите степень интерполяционного многочлена n ≤ 19:  19



Узлы для построения P_19(x):
x_0 = 0.000000, f(x_0) = 0.000000
x_1 = -0.052632, f(x_1) = -0.053992
x_2 = -0.105263, f(x_2) = -0.110609
x_3 = -0.157895, f(x_3) = -0.169705
x_4 = -0.210526, f(x_4) = -0.231135
x_5 = -0.263158, f(x_5) = -0.294757
x_6 = -0.315789, f(x_6) = -0.360428
x_7 = -0.368421, f(x_7) = -0.428010
x_8 = -0.421053, f(x_8) = -0.497364
x_9 = -0.473684, f(x_9) = -0.568356
x_10 = -0.526316, f(x_10) = -0.640855
x_11 = -0.578947, f(x_11) = -0.714733
x_12 = -0.631579, f(x_12) = -0.789866
x_13 = -0.684211, f(x_13) = -0.866133
x_14 = -0.736842, f(x_14) = -0.943421
x_15 = -0.789474, f(x_15) = -1.021617
x_16 = -0.842105, f(x_16) = -1.100617
x_17 = -0.894737, f(x_17) = -1.180321
x_18 = -0.947368, f(x_18) = -1.260635
x_19 = -1.000000, f(x_19) = -1.341471

Форма Лагранжа:
P_19_L(1.0) = 0.201457977294922
f(1.0) = 0.341470984807897 сама функция
Абсолютная погрешность: |f(x) - P_n_L(x)| = 0.140013007512975
Сумма коэффициентов : -35345263797.927658081054688

Форма Ньютона:
P_19_N(1.0) = 

In [4]:
import math

def f_d(d, r, rho):
    'Функция'
    return (math.pi / 3) * (d**3 - 3*d**2*r + 4*r**3*rho)

def f_d_derivative(d, r, rho):
    ''
    return (math.pi / 3) * (3*d**2 - 6*d*r)

def newton_method(f, f_deriv, x0, args=(), tol=1e-8, max_iter=100):
    """Метод Ньютона (уточнение корней)"""
    x = x0
    for i in range(max_iter):
        fx = f(x, *args)
        if abs(fx) < tol:
            return x, i + 1
        
        fpx = f_deriv(x, *args)
        if abs(fpx) < 1e-15:
            break
            
        x = x - fx / fpx
        
    return x, max_iter

def solve_ball_problem():
    """Задача о погружении шара"""
    materials = {
        'Пробка': 0.25,
        'Бамбук': 0.4, 
        'Сосна (белая)': 0.5,
        'Кедр': 0.55,
        'Дуб': 0.7,
        'Бук': 0.75,
        'Красное дерево': 0.8,
        'Тиковое дерево': 0.85,
        'Парафин': 0.9,
        'Лёд/Полиэтилен': 0.92,
        'Пчелиный воск': 0.95
    }
    
    r_cm = float(input("Введите радиус шара в см: "))
    r = r_cm / 100  # переводим в метры
    
    print(f"\nРезультаты для шара радиуса r = {r_cm} см = {r} м")
    print("=" * 65)
    print(f"{'Материал':<20} {'ρ, г/мл':<12} {'d, м':<12} {'d, см':<12} {'Итерации количество':<12}")
    
    for material, density in materials.items():
        rho = density  # относительная плотность
        
        x0 = r  # начинаем с половины погружения
        d, iterations = newton_method(f_d, f_d_derivative, x0, args=(r, rho))
        d_cm = d * 100
        
        print(f"{material:<20} {density:<12.2f} {d:<12.4f} {d_cm:<12.2f} {iterations:<12}")
    
if __name__ == "__main__":
    solve_ball_problem()

Введите радиус шара в см:  3



Результаты для шара радиуса r = 3.0 см = 0.03 м
Материал             ρ, г/мл      d, м         d, см        Итерации количество
Пробка               0.25         0.0196       1.96         3           
Бамбук               0.40         0.0260       2.60         3           
Сосна (белая)        0.50         0.0300       3.00         1           
Кедр                 0.55         0.0320       3.20         2           
Дуб                  0.70         0.0382       3.82         3           
Бук                  0.75         0.0404       4.04         3           
Красное дерево       0.80         0.0428       4.28         4           
Тиковое дерево       0.85         0.0453       4.53         4           
Парафин              0.90         0.0483       4.83         4           
Лёд/Полиэтилен       0.92         0.0496       4.96         4           
Пчелиный воск        0.95         0.0519       5.19         5           
